In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.python.ops.gen_math_ops import Exp
from datetime import datetime
import os
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import time
import random
import sys 

from model.actor import Actor
from model.critic import Critic
from model.enviroment import StimulateEnv
from model.ddpg import DDPG 
from model.embedding import VideoGenreEmbedding, UserVideoEmbedding
from model.ou_noise import OUNoise

In [2]:
hub = '/Users/minhtuan/Documents/Documents/Work/Hanoi/Hub'
path = '/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/X/movielens/airflow_folder/drl_melody/storage/dataset1M'

## RUN USER

In [7]:
PATH_USER_DICT = os.path.join(hub, "user_dict.npy")
PATH_EVAL_DATSET = os.path.join(hub, "eval_dict.npy")
PATH_USER_HISTORY_LENS = os.path.join(hub, 'users_history_len.npy')
PATH_DATA_NUMBER = os.path.join(hub, "data_number.npy")

users_dict = np.load(PATH_USER_DICT,allow_pickle='TRUE').item()
eval_users_dict = np.load(PATH_EVAL_DATSET,allow_pickle='TRUE').item()
data_number = np.load(PATH_DATA_NUMBER,allow_pickle='TRUE').item()
users_history_lens = np.load(PATH_USER_HISTORY_LENS, allow_pickle='TRUE')
all_items = {data[0] for i, k in users_dict.items() for data in k}  ## list video toan tap data 

user_dataset = eval_users_dict
user_id = 4834

users_history_lens = round(len(user_dataset[user_id]) * 0.6) ## split data in EVAL into 2 pieces: one for history and other for streaming 
watched_videos = [data[0] for data in user_dataset[user_id]][:users_history_lens]
user_dict_history = {user_id: eval_users_dict[user_id][:users_history_lens]}
# newest_watched_video = np.random.choice([i[0] for i in user_dataset[user_id]])
newest_watched_video = [data[0] for data in user_dataset[user_id]][users_history_lens:][0]
items_ids = np.array(list(set(all_items) - set(watched_videos)))
len_items_ids = len(items_ids)
users_num = data_number['users_num']
items_num = data_number['items_num']

STATE_SIZE =  5 #len_items_ids ## 1445 là số lượng videos sau khi trừ đi các video đã xem trong history
num_actions =  5 #len_items_ids ## Number of list video to be choosed
output_dim = 5 #len_items_ids

env_prod = StimulateEnv(user_id, newest_watched_video, user_dict_history, users_history_lens, STATE_SIZE)
recommender  = DDPG(env_prod, users_num, items_num, num_actions, STATE_SIZE, output_dim)  # output_dim là output của State_emebedding, để 1445 vì đầu vào của actor.evaluate_actor là (1445,400)


EMBEDDING_SIZE = 100
epsilon_for_priority = 1e-6
batch_size = 32
exploration_noise = OUNoise(num_actions)

## item_ids should comes from history of user 

# x = items_ids
x = watched_videos[- STATE_SIZE:]
array_x = np.reshape(x,[1, num_actions])
state_value = tf.convert_to_tensor(x, dtype=tf.float32)
state_value = tf.expand_dims(state_value, axis=0)

action = recommender.evaluate_actor(state_value) ## create a ranking video weight 
noise = exploration_noise.noise()
action = action[0] + noise

recommended_item = recommender.recommend_item(action, all_items, env_prod.old_watched, top_k= 5) ## create a list of suggested video 
next_items_ids_embs, reward, done, _ = env_prod.step(recommended_item)

array_x_next = np.reshape(next_items_ids_embs,[1, num_actions])
state_value_next = tf.convert_to_tensor(next_items_ids_embs, dtype=tf.float32)
state_value_next = tf.expand_dims(state_value_next, axis=0)

reward = np.sum(reward)

recommender.add_experience(state_value, state_value_next, action, reward, done)


/Users/minhtuan/anaconda3/envs/hanoi/lib/python3.10/site-packages/keras/saving/saving_lib.py:510: UserWarning: Could not load weights in object <keras.layers.core.embedding.Embedding object at 0x17755e320>. Skipping object. Exception encountered: Layer 'user_embedding' expected 0 variables, but received 1 variables during loading. Expected: []
  _load_state(
/Users/minhtuan/anaconda3/envs/hanoi/lib/python3.10/site-packages/keras/saving/saving_lib.py:510: UserWarning: Could not load weights in object <keras.layers.core.embedding.Embedding object at 0x17755df60>. Skipping object. Exception encountered: Layer 'video_embedding' expected 0 variables, but received 1 variables during loading. Expected: []
  _load_state(
/Users/minhtuan/anaconda3/envs/hanoi/lib/python3.10/site-packages/keras/saving/saving_lib.py:510: UserWarning: Could not load weights in object <keras.layers.core.dense.Dense object at 0x17755d990>. Skipping object. Exception encountered: Layer 'dense_4' expected 0 variables, 

#### Note flow 

reset will take out 5 videos from history of user, 5 videos 
        -> embedding -> state -> actor.network 
        -> action (ranking video base on user) -> feed on recommender.recommend_item (take out video to suggest) 

In [8]:
recommended_item, env_prod.old_watched, newest_watched_video, x

(array([1135, 3515, 1434, 2178, 1544]),
 [2395, 2762, 2724, 314, 2702],
 2840,
 [2395, 2762, 2724, 314, 2702])

1/9: finish flow evaluate_actor and predict

next step: replay_buffer and train sections

## TRAIN SECTION

In [3]:
PATH_USER_DICT = os.path.join(hub, "user_dict.npy")
PATH_EVAL_DATSET = os.path.join(hub, "eval_dict.npy")
PATH_USER_HISTORY_LENS = os.path.join(hub, 'users_history_len.npy')
PATH_DATA_NUMBER = os.path.join(hub, "data_number.npy")

users_dict = np.load(PATH_USER_DICT,allow_pickle='TRUE').item()
eval_users_dict = np.load(PATH_EVAL_DATSET,allow_pickle='TRUE').item()
data_number = np.load(PATH_DATA_NUMBER,allow_pickle='TRUE').item()
users_history_lens = np.load(PATH_USER_HISTORY_LENS, allow_pickle='TRUE')
all_items = {data[0] for i, k in users_dict.items() for data in k}  ## list video toan tap data 

user_dataset = eval_users_dict
users_num = data_number['users_num']
items_num = data_number['items_num']

STATE_SIZE = 20
num_actions = 20
output_dim = 5

user_id = 4833
users_history_lens = round(len(eval_users_dict[user_id]) * 0.6)
watched_videos = [video[0] for video in eval_users_dict[user_id]][:users_history_lens]
newest_watched_video = [video[0] for video in eval_users_dict[user_id]][users_history_lens:][0]
enviroment = StimulateEnv(user_id, newest_watched_video, eval_users_dict, users_history_lens,STATE_SIZE)

In [7]:

#specify parameters here:
episodes=1

#Randomly initialize critic,actor,target critic, target actor network  and replay buffer   
agent  = DDPG(enviroment, users_num, items_num, num_actions, STATE_SIZE, output_dim)  # output_dim là output của State_emebedding, để 1445 vì đầu vào của actor.evaluate_actor là (1445,400)
exploration_noise = OUNoise(num_actions)
counter=0
reward_per_episode = 0    
total_reward=0
#saving reward:
reward_st = np.array([0])
    
agent  = DDPG(enviroment, users_num, items_num, num_actions, STATE_SIZE, output_dim)  # output_dim là output của State_emebedding, để 1445 vì đầu vào của actor.evaluate_actor là (1445,400)

for i in range(0, episodes):
    print("==== Starting episode no:",i,"====","\n")
    user_id, watched_videos, done = enviroment.reset()

    # user_id = 4833
    # users_history_lens = round(len(eval_users_dict[user_id]) * 0.6)
    users_history_lens = STATE_SIZE
    eval_users_dict_stimulated = {user_id: eval_users_dict[user_id][:users_history_lens]}
    watched_videos = [video[0] for video in eval_users_dict[user_id]][:users_history_lens]
    newest_watched_video = [video[0] for video in eval_users_dict[user_id]][users_history_lens:]
    old_watched = enviroment.old_watched
    newest_watched_video_ = newest_watched_video[i] ## initiate video to start a session

    # users_history_lens =  round(len(user_dataset[user_id]) * 0.6)
    # watched_videos =  [data[0] for data in eval_users_dict[user_id]][:users_history_lens]
    reward_per_episode = 0
    steps = len(eval_users_dict[user_id][users_history_lens:])
    for t in range(0, steps-1):
        x = watched_videos[- STATE_SIZE:]
        ## change shape to fit evaluate_actor 
        state_value = tf.convert_to_tensor(x, dtype=tf.float32)
        state_value = tf.expand_dims(state_value, axis=0)
        action = agent.evaluate_actor(np.reshape(state_value,[1, num_actions]))
        noise = exploration_noise.noise()
        action = action[0] + noise #Select action according to current policy and exploration noise
        # print("Action at step", t ," :",action,"\n")
        enviroment = StimulateEnv(user_id, newest_watched_video_, eval_users_dict_stimulated, users_history_lens, STATE_SIZE)
        recommended_item = agent.recommend_item(action, all_items, old_watched, top_k= STATE_SIZE)

        next_items_ids_embs, reward, done, _= enviroment.step(recommended_item)
        print("history to embs:", x)
        print("env items:", enviroment.items)
        print("suggested items:", recommended_item)
        print("actually watch", newest_watched_video_)
        print("next items to embs:", next_items_ids_embs)
        # print(newest_watched_video_, recommended_item, next_items_ids_embs)
        state_value_next = tf.convert_to_tensor(next_items_ids_embs, dtype=tf.float32)
        state_value_next = tf.expand_dims(state_value_next, axis=0)
        state_value_next = np.reshape(state_value_next,[1, num_actions])
        
        newest_watched_video_ = newest_watched_video[t+1] ## giả lập truyền liên tục vào Enviroment các video mới cho lần kế tiếp
        old_watched = np.append(old_watched, enviroment.newest_watched_video)
        list_watched_old = eval_users_dict_stimulated[user_id]
        list_watched_old.append((enviroment.newest_watched_video, 5))
        eval_users_dict_stimulated = {user_id: list_watched_old} 
        watched_videos.append(newest_watched_video[t])
        
        #add s_t,s_t+1,action,reward to experience memory
        agent.add_experience(state_value, state_value_next, action, reward, done)
        #train critic and actor network
        if counter > 30: 
            agent.train()
        reward_per_episode+=reward[0]
        counter+=1
        #check if episode ends:
        if (done or (t == steps-1) or t == 30):
            print('EPISODE: ',i,' Steps: ',t,' Total Reward: ',reward_per_episode)
            print("Printing reward to file")
            exploration_noise.reset() #reinitializing random noise for action exploration
            reward_st = np.append(reward_st,reward_per_episode)
            np.savetxt('episode_reward.txt',reward_st, newline="\n")
            print('\n\n')
            break
total_reward+=reward_per_episode            
print("Average reward per episode {}".format(total_reward / episodes)) 


/Users/minhtuan/anaconda3/envs/hanoi/lib/python3.10/site-packages/keras/saving/saving_lib.py:510: UserWarning: Could not load weights in object <keras.layers.core.embedding.Embedding object at 0x1761f7b80>. Skipping object. Exception encountered: Layer 'user_embedding' expected 0 variables, but received 1 variables during loading. Expected: []
  _load_state(
/Users/minhtuan/anaconda3/envs/hanoi/lib/python3.10/site-packages/keras/saving/saving_lib.py:510: UserWarning: Could not load weights in object <keras.layers.core.embedding.Embedding object at 0x1761f5c90>. Skipping object. Exception encountered: Layer 'video_embedding' expected 0 variables, but received 1 variables during loading. Expected: []
  _load_state(
/Users/minhtuan/anaconda3/envs/hanoi/lib/python3.10/site-packages/keras/saving/saving_lib.py:510: UserWarning: Could not load weights in object <keras.layers.core.dense.Dense object at 0x1761f4040>. Skipping object. Exception encountered: Layer 'dense_12' expected 0 variables,

==== Starting episode no: 0 ==== 

[   1    2    3 ... 3950 3951 3952]
history to embs: [1929, 3108, 10, 296, 858, 3503, 1221, 3307, 2858, 1193, 912, 923, 2019, 3134, 3224, 1207, 1237, 3030, 908, 2303]
env items: [1929, 3108, 10, 296, 858, 3503, 1221, 3307, 2858, 1193, 912, 923, 2019, 3134, 3224, 1207, 1237, 3030, 908, 2303]
suggested items: [2589  588 3665  943   17 2081 1080 3249 2661 3801 3868  128 2534 1911
 3540 1814  717 1599 2865 3932]
actually watch 1084
next items to embs: [1929, 3108, 10, 296, 858, 3503, 1221, 3307, 2858, 1193, 912, 923, 2019, 3134, 3224, 1207, 1237, 3030, 908, 2303]
[   1    2    3 ... 3950 3951 3952]
history to embs: [3108, 10, 296, 858, 3503, 1221, 3307, 2858, 1193, 912, 923, 2019, 3134, 3224, 1207, 1237, 3030, 908, 2303, 1084]
env items: [3108, 10, 296, 858, 3503, 1221, 3307, 2858, 1193, 912, 923, 2019, 3134, 3224, 1207, 1237, 3030, 908, 2303, 1084]
suggested items: [1951 2017 1468  600 3370 2534 2658 2381 3098 2143 2131 1701  892 2523
 1015 2271 2981 122

### stuck: next_state in replay_memory not work properly vì hiện tại đang lấy là các videos từ lịch sử tiếp theo. Nghiên cứu lại vấn đề next_state

Hiện tại đang là, history -> embedding state (neural of actor): action -> nhân action * list video tổng (đã bỏ qua history) -> weight các videos -> Lấy 20 video có weight tốt nhát -> compare with thực tế -> lấy thông tin next_state từ history -> Lưu vào trong replay_memory

In [8]:
eval_users_dict[user_id], old_watched, newest_watched_video_, eval_users_dict_stimulated[user_id], watched_videos

([(480, 4),
  (2269, 2),
  (1210, 5),
  (2090, 4),
  (1586, 1),
  (3208, 2),
  (593, 4),
  (527, 5),
  (1704, 3),
  (608, 1),
  (3006, 3),
  (1617, 4),
  (457, 5),
  (17, 4),
  (2571, 5),
  (497, 4),
  (150, 5),
  (590, 3),
  (1921, 1),
  (2268, 4),
  (2336, 3),
  (553, 4),
  (3476, 2),
  (508, 4),
  (356, 4),
  (34, 4),
  (2501, 5),
  (588, 4),
  (2916, 5),
  (2353, 5),
  (2355, 4),
  (1641, 3),
  (1909, 4),
  (2617, 5),
  (1198, 5),
  (266, 2)],
 array([2353, 2355, 1641, 1909, 2617,  527,  593,  527, 1704,  608]),
 3006,
 [(480, 4),
  (2269, 2),
  (1210, 5),
  (2090, 4),
  (1586, 1),
  (527, 5),
  (593, 5),
  (527, 5),
  (1704, 5),
  (608, 5)],
 [480, 2269, 1210, 2090, 1586, 3208, 593, 527, 1704, 608])

In [29]:
agent.replay_memory

deque([(<tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[3098.,  908., 2143., 1385.,  356.]], dtype=float32)>,
        array([[3098.,  908., 2143., 1385.,  356.]], dtype=float32),
        <tf.Tensor: shape=(5,), dtype=float32, numpy=
        array([-0.13678792, -0.61636907, -0.32507613, -0.31683338,  0.03700582],
              dtype=float32)>,
        [-0.5],
        False),
       (<tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[ 908., 2143., 1385.,  356., 1035.]], dtype=float32)>,
        array([[ 908., 2143., 1385.,  356., 3526.]], dtype=float32),
        <tf.Tensor: shape=(5,), dtype=float32, numpy=
        array([-0.2489828 , -0.70440966, -0.30874968,  0.14436762, -0.02266764],
              dtype=float32)>,
        [-0.5],
        False),
       (<tf.Tensor: shape=(1, 5), dtype=float32, numpy=array([[2143., 1385.,  356., 1035., 3526.]], dtype=float32)>,
        array([[2143., 1385.,  356., 3526., 3526.]], dtype=float32),
        <tf.Tensor: shape=(5,), dtype=float32,